# Requirements

In [ ]:
pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model Set

In [ ]:
from urllib import request
import requests
from bs4 import BeautifulSoup 
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus # 한글 처리용 함수
import pandas as pd
import numpy as np
import time
import os
from konlpy.tag import Okt
from googletrans import Translator
import torch
import nltk
from torch.nn import functional as F

# Flask 구동

In [ ]:
!mkdir /root/.ngrok2 && echo 'authtoken: 2K7I7O1DpP2jRhLZkbgOyok0X8H_6ZVLBrT25o6NA81BJMC9a' > /root/.ngrok2/ngrok.yml

In [ ]:
# 뉴스 추출 메서드, Daum News 기반
def get_news(url= 'https://news.daum.net/'):
  header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

  titles = []

  html_page = requests.get(url, header)
  soup = BeautifulSoup(html_page.text, 'lxml') 
  document = soup.select(".link_txt")  # select_one 는 한개만 받아옴

  for t in document:
    link = t.attrs['href']
    if link[:20] == 'https://v.daum.net/v' : # 다음 뉴스 링크만 가져오기
      titles.append({'title': t.text, 'link': link})
  return titles

In [ ]:
# 뉴스 본문 추출, Daum News 기반
def get_news_text(link= '') :
  news_text = ''
  sub_html = requests.get(link, header)
  sub_soup = BeautifulSoup(sub_html.text, 'lxml')
  contents = sub_soup.select('div.news_view.fs_type1 > div.article_view > section > p')[:-1]  
  for p in contents:
    news_text+=(p.text)   
  return news_text

In [ ]:
from flask import Flask, url_for, redirect, render_template, request
from flask_ngrok import run_with_ngrok
import os

path = '/content/drive/MyDrive/3차 프로젝트_Early Bird팀/은아/templates'

app = Flask(__name__, template_folder=path)
run_with_ngrok(app)

titles = get_news()

@app.route('/')

def index():
    return render_template('index.html', news_lists = titles)
    
# report 페이지에 주요 변수들과 함수들을 정의해 준다.

@app.route('/report',methods=['POST','GET'])
def report():
    if request.method == 'POST':
        temp = request.form['id_summary']
    else:
        temp = None

    news_link = request.args.get('news_link')
    news_title = request.args.get('news_title')
    news_text = get_news_text(news_link)

# 사용될 모든 변수들을 render_template에 정의해주어야 html에서 작동할 수 있다. 
    return render_template('report.html', 
                           news_title= news_title, 
                           news_link = news_link,
                           news_text = news_text,
                           text = temp
                           )
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1b62-34-73-158-124.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [08/Feb/2023 08:29:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2023 08:29:35] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Feb/2023 08:29:38] "GET /report?news_link=https%3A%2F%2Fv.daum.net%2Fv%2F20230208115033528&news_title=%0A++++++++++++++++++++++++++++++++++++%22전북+암+발생률+두+번째%22+발표에+%22통계+착시%22+반박%0A++++++++++++++++++++++++++++++++ HTTP/1.1" 200 -
ERROR:__main__:Exception on /report [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise val